In [ ]:
import os
import sys
import logging
import json

from getpass import getuser
from datetime import datetime

from side_function import create_folder_output, move_input_files_to_output_folder

In [2]:
from paths import path_folder_typology, path_folder_construction_and_load_library, path_file_gis, path_file_epw,\
    path_folder_simulation_parameter, path_simulation_parameter, path_folder_output, path_energyplus_exe,\
    path_folder_simulation, path_folder_context_hbjson, path_folder_building_simulation, path_logger,\
    path_LBT_user_defined,path_input_file
from paths  import unit_gis, target_buildings, VF_criterion_shading

# create the simulation folder and move the inputs to the output folder
create_folder_output(path_folder_simulation=path_folder_simulation)
move_input_files_to_output_folder(path_folder_simulation, path_EP_parameter_par=path_folder_simulation_parameter, path_sim_input_par=path_input_file,
                                      path_epw_par=None, path_gis_par=None)
# logging.basicConfig(filename=path_logger, format='%(asctime)s %(levelname)s %(message)s', filemode='w')
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[logging.FileHandler(path_logger), logging.StreamHandler(sys.stdout)])
# log #
logging.info("Input data extracted and simulation folder created")
#

2022-07-12 12:56:23,902 [INFO] Input data extracted and simulation folder created


In [3]:
print(path_folder_building_simulation)

D://Elie//PhD//Simulation\Program_output\Simulation__111_12_07_2022_12h_56m_22s\Buildings


In [4]:
from library import Library

## load user defined object for the LBT, before importing the LBT (otherwise the objects will not be updated
Library.load_HBjson_sets(path_construction_and_load_library_folder=path_folder_construction_and_load_library,
                         path_LBT_user_defined=path_LBT_user_defined)
# log #
logging.info("Library imported")
#

2022-07-12 12:56:23,954 [INFO] Library imported


In [5]:
from urban_canopy.urban_canopy import Urban_canopy


2022-07-12 12:56:24,373 [INFO] Successfully imported Honeybee plugin: honeybee_energy
2022-07-12 12:56:24,442 [INFO] Successfully imported Ladybug plugin: ladybug_comfort
2022-07-12 12:56:24,442 [INFO] Successfully imported Ladybug plugin: ladybug_geometry
2022-07-12 12:56:24,503 [INFO] Successfully imported Honeybee plugin: honeybee_radiance
2022-07-12 12:56:24,503 [INFO] Successfully imported Honeybee plugin: honeybee_schema
2022-07-12 12:56:24,503 [INFO] Successfully imported Honeybee plugin: honeybee_standards
2022-07-12 12:56:24,909 [INFO] Successfully imported Honeybee-energy plugin: honeybee_energy_standards
2022-07-12 12:56:24,973 [INFO] Successfully imported Dragonfly plugin: dragonfly_energy
2022-07-12 12:56:24,975 [INFO] Successfully imported Dragonfly plugin: dragonfly_schema
2022-07-12 12:56:24,983 [INFO] Successfully imported Dragonfly plugin: dragonfly_uwg


In [6]:
## Create Urban Canopy
U_c = Urban_canopy("random_neighborhood")

## Load Typologies in the Urban canopy object
U_c.load_typologies(path_folder_typology)

Typology "BER_proj_R0-W0-G1" does not have building_zon type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G2" does not have building_zon type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G3" does not have building_zon type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G4" does not have building_zon type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G5" does not have building_zon type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G6" does not have building_zon type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G7" does not have building_zon type, it will be set as "residential" by default
Typology "BER_proj_R0-W1-G0" does not have building_zon type, it will be set as "residential" by default
Typology "BER_proj_R0-W2-G0" does not have building_zon type, it will be set as "residential" by default
Typology "BER_proj_R1-W0-G0" does not have building_zon

In [7]:
## Extract data from GIS files
U_c.extract_gis_2D(path_file_gis, unit_gis)


# log #
logging.info("GIS extracted")
#


2022-07-12 12:56:25,073 [INFO] GIS extracted


In [8]:
## Define buildings to simulate
U_c.select_target_building(target_buildings)

In [9]:
## Create Ladybug geometries with GIS footprint
U_c.create_building_LB_geometry_footprint()
## Create HB room envelop with GIS footprint
U_c.create_building_HB_room_envelop()

In [10]:
# filter context and identify the buildings to simulate

U_c.filter_context(0.1)

In [11]:
print(len(U_c.building_dict[0].context_buildings_id))

0


In [12]:
U_c.building_dict[0].typology = U_c.typology_dict["BER_proj_R1-W1-G4_s_005"]

In [13]:
# print(U_c.building_dict[0].num_floor)

In [14]:
# convert to DF stories and buildings
U_c.create_DF_building_according_to_typology()
# convert to HB model
U_c.generate_HB_model()
# U_c.convert_DF_building_to_HB_models()
U_c.HB_solve_adjacencies()

In [15]:
## Force rotation on building
# need to guess it and rotate it at the beginning

U_c.building_dict[0].HB_model_force_rotation(180)

In [16]:
# create windows
U_c.assign_conditioned_zone()
U_c.HB_building_window_generation_floor_area_ratio()

# add shades
U_c.add_context_surfaces_to_HB_model()
# assign constructions, loads etc...
U_c.apply_buildings_characteristics()

# add blinds

# add thermal mass
U_c.add_thermal_mass_int_wall()

# log #
logging.info("Building modeled")
#

2022-07-12 12:56:25,280 [INFO] Building modeled


In [17]:
print(U_c.building_dict[0].HB_model.rooms[0].properties.energy.program_type)
print(U_c.building_dict[0].HB_model.rooms[0].properties.energy.internal_masses[0].construction)


Program Type: IS_Residential_5282_ref_AB
Construction,
 IS_InternalWall_Internalmass_Res_Ref, !- name
 IS_CB41-4-10x50x20-030_half, !- layer 1
 IS_CementMortar_0.015;    !- layer 2


In [18]:
U_c.create_simulation_folder_buildings(path_folder_building_simulation)

In [19]:
# All the buildings that are target buildings
U_c.context_to_hbjson(path_folder_context_hbjson)
U_c.context_surfaces_to_hbjson(path_folder_building_simulation)
# U_c.GIS_context_individual_to_hbjson(path_folder_building_simulation)

In [20]:
## Merge simulation parameters files
U_c.load_simulation_parameter(path_folder_simulation_parameter, path_simulation_parameter)
## Add design days
U_c.add_design_days_to_simulation_parameters(path_simulation_parameter, path_file_epw)



In [21]:

#### Extract simulation parameters ####
U_c.model_to_HBjson(path_folder_building_simulation)




In [ ]:
#### Extract simulation parameters ####
U_c.simulate_idf(path_folder_building_simulation, path_simulation_parameter, path_file_epw, path_energyplus_exe)







KeyboardInterrupt: 